# 인터파크도서 베스트셀러

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

In [72]:
base_url = 'https://book.interpark.com'
sub_url = '/display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028#'
url = base_url + sub_url
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

## 1. 찾으려고 하는 목록 데이터

In [142]:
lis = soup.select('.rankBestContentList > ol > li')

In [143]:
len(lis)

15

### 하나의 데이터로 정보 추출

In [123]:
li = lis[11]

In [125]:
#순위 
#1~9위
li.select_one('.cover .rankNumber span')['class'][1][-1]

2

In [110]:
#10~15위
element = '<div class="rankNumber digit2"><span class="rankBtn_ctrl rkNum_M01"></span><span class="rankBtn_ctrl rkNum_M02"></span>'
mini_soup = BeautifulSoup(element, 'html.parser')
mini_soup

<div class="rankNumber digit2"><span class="rankBtn_ctrl rkNum_M01"></span><span class="rankBtn_ctrl rkNum_M02"></span></div>

In [111]:
mini_soup.select('.rankBtn_ctrl')

[<span class="rankBtn_ctrl rkNum_M01"></span>,
 <span class="rankBtn_ctrl rkNum_M02"></span>]

In [131]:
rank = ''
for i in range(len(li.select('.cover .rankNumber span'))):
    rank += li.select('.cover .rankNumber span')[i]['class'][1][-1]
rank

'12'

In [81]:
#제목
title = li.select_one('.itemName > strong').text.strip()

In [9]:
#서브페이지의 주소
href = li.find('a')['href']
href

'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=356179385&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [10]:
sub_req = requests.get(base_url + href)
sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
sub_soup


<!DOCTYPE html>

<html lang="ko">
<head>
<title>싸니까 믿으니까 인터파크도서</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="인터파크도서, 인터넷 서점, eBook, 비스킷탭, 음반, DVD, 중고책, 중고책서점, 신간도서, 국내도서, 외국도서, 추천도서" name="keywords"/>
<meta content="문학은 인터파크 도서!" name="description"/>
<meta content="no-cache" http-equiv="Cache-Control">
<meta content="0" http-equiv="Expires">
<meta content="no-cache" http-equiv="Pragma">
<link href="https://sslimage.interpark.com/interpark.ico" rel="shortcut icon"/>
<meta content="no" http-equiv="imagetoolbar">
<script type="text/javascript">
        var isFancyAlert = false;    // IMFS에서는 msgAlert 기능을 비활성화시킴
        var imfsSSLDomain = "https://book.interpark.com"; // com.interpark.util.web.ServletUtils.IMFS_URL_SSL 값을 사용하였으나, Fix 처리 한듯.
    </script>
<script src="/static/js/common_ssl.js" type="text/javascript"></script>
<script src="/static/js/CookieUtil.js" type="text/java

In [34]:
info_list = sub_soup.select('.bookInfoBox li')
info_list

[<li>저 : <a href="#" onclick='fn_wiseLogRedirect("infor","basic_inform","writer","http://bsearch.interpark.com/dsearch/book.jsp?sch=all&amp;sc.shopNo=&amp;bookblockname=s_main&amp;booklinkname=s_main&amp;query=김혜자","_blank");return false;'>김혜자</a></li>,
 <li>출판사 : <a href="#" id="hdelvMafcEntrNm" onclick='fn_wiseLogRedirect("infor","basic_inform","publisher","http://bsearch.interpark.com/dsearch/book.jsp?sch=all&amp;query=수오서재","_blank");return false;'>수오서재</a></li>,
 <li>발행 : 2022년 12월 22일</li>,
 <li>쪽수 : 376</li>,
 <li>ISBN : 9791190382915</li>]

In [40]:
info_list[0].text

'저 : 김혜자'

In [41]:
# 서브페이지에서 필요한 정보 추출
info_dict = {}
for li in info_list:
    info = li.text.split(':')
    key = info[0].strip()
    value = info[1].strip()
    info_dict[key] = value

In [42]:
info_dict

{'저': '김혜자',
 '출판사': '수오서재',
 '발행': '2022년 12월 22일',
 '쪽수': '376',
 'ISBN': '9791190382915'}

In [51]:
author = info_dict['저']
try:
    editor = info_dict['역']
except:
    editor = '-'
publisher = info_dict['출판사']
pages = int(info_dict['쪽수'].split(' ')[0].replace(',', ''))
date = info_dict['발행']

price = sub_soup.select_one('p.tt_price .price').text.replace(',','')

title, author, editor, publisher, pages, date, price

('생에 감사해', '김혜자', '-', '수오서재', 376, '2022년 12월 22일', '15300')

## 2. 당일 베스트셀러 정보를 가져오기

In [148]:
lines = []
for li in lis:
    #순위
    rank = ''
    for i in range(len(li.select('.cover .rankNumber span'))):
        rank += li.select('.cover .rankNumber span')[i]['class'][1][-1]
    #책 제목
    title = li.select_one('.itemName').text
    #sub page
    href = li.find('a')['href']
    sub_req = requests.get(base_url + href)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
    #필요한 정보만 가져오기
    info_list = sub_soup.select('.bookInfoBox li')
    #딕셔너리 형태로 저장
    info_dict = {}
    for li in info_list:
        info = li.text.split(':')
        key = info[0].strip()
        value = info[1].strip()
        info_dict[key] = value
    
    #저자
    author = info_dict['저']
    #역은이(없으면 '-'로 처리)
    try:
        editor = info_dict['역']
    except:
        editor = '-'
    #출판사
    publisher = info_dict['출판사']
    #쪽수
    pages = int(info_dict['쪽수'].split(' ')[0].replace(',', ''))
    #발행일
    date = info_dict['발행']
    #책 가격
    price = sub_soup.select_one('p.tt_price .price').text.replace(',','')
    
    lines.append({'순위' : int(rank), '제목' : title, '저자' : author, '역자' : editor, '출판사' : publisher, '쪽수' : pages, '발행일' : date, '가격' : price})

In [149]:
df = pd.DataFrame(lines)

In [150]:
df

,순위,제목,저자,역자,출판사,쪽수,발행일,가격
0,1,생에 감사해,김혜자,-,수오서재,376,2022년 12월 22일,15300
1,2,만일 내가 인생을 다시 산다면(10만 부 기념 스페셜 에디션),"김혜남, 김혜남",-,메이븐,280,2022년 11월 11일,15480
2,3,미스터 프레지던트,탁현민,-,메디치미디어,436,2023년 01월 18일,19800
3,4,안젤리크,기욤 뮈소,양영란,밝은세상,0,2022년 12월 21일,15120
4,5,원씽(The One Thing)(리커버 특별판),"게리 켈러, 제이 파파산",구세희,비즈니스북스,280,2013년 08월 30일,12600
5,6,스카이 버스,분당강쌤,-,다산에듀,280,2023년 01월 03일,16200
6,6,트렌드 코리아 2023,"김난도, 김난도, 추예린, 전미영, 전미영, 최지혜, 이수진, 권정윤, 이준영, 이...",-,미래의창,424,2022년 10월 05일,17100
7,8,빅 히스토리,"데이비드 크리스천, 신시아 브라운, 크레이그 벤저민",-,웅진지식하우스,640,2022년 12월 23일,29700
8,8,이토록 평범한 미래,김연수,-,문학동네,276,2022년 10월 07일,12600
9,8,아버지의 해방일지,정지아,-,창비,268,2022년 09월 02일,13500
